# Cosmos Video Generation Client - Interactive Example

This notebook demonstrates how to use the Cosmos Video Client to generate videos from text prompts and optional images.

## Setup and Imports

In [ ]:
# Import the client
from cosmos_client import CosmosVideoClient
import os
from IPython.display import Video, Image, display
from PIL import Image as PILImage, ImageDraw
import requests
import time

## Initialize Client and Check Service

In [ ]:
# Initialize the client
client = CosmosVideoClient("http://localhost:8001")

# Check if service is running
try:
    health = client.health_check()
    print("✅ Cosmos service is running!")
    print(f"Model: {health['cosmos_config']['model_size']}")
    print(f"GPUs: {health['cosmos_config']['num_gpus']}")
    print(f"Output directory: {health['output_dir']}")
except Exception as e:
    print(f"❌ Service not available: {e}")
    print("Make sure to start the service with:")
    print("uvicorn cosmos_video_service:app --port 8001")

## Example 1: Generate Video from Text Only

In [ ]:
# Simple text-to-video generation
prompt = "A robot arm picking up a red apple from a wooden table"

print(f"Generating video with prompt: {prompt}")
print("This may take a few minutes...")

result = client.generate_video_openai_style(prompt=prompt)

if result["success"]:
    print("✅ Video generated successfully!")
    print(f"Video URL: {result['video_url']}")
    
    # Download and display the video
    video_path = "/tmp/example1_video.mp4"
    if client.download_video(result['full_url'], video_path):
        print(f"Video saved to: {video_path}")
        # Display in notebook
        display(Video(video_path, width=400))
else:
    print(f"❌ Generation failed: {result['error']}")

## Example 2: Create a Test Image and Generate Video

In [ ]:
# Create a simple test image
def create_test_image():
    img = PILImage.new('RGB', (512, 512), color='lightblue')
    draw = ImageDraw.Draw(img)
    
    # Draw a simple scene
    # Table
    draw.rectangle([50, 350, 450, 400], fill='brown', outline='black', width=2)
    
    # Apple
    draw.ellipse([200, 300, 250, 350], fill='red', outline='darkred', width=2)
    
    # Robot arm (simplified)
    draw.rectangle([300, 200, 320, 300], fill='gray', outline='black', width=2)
    draw.rectangle([290, 280, 330, 300], fill='darkgray', outline='black', width=2)
    
    return img

# Create and save the test image
test_img = create_test_image()
test_img_path = "/tmp/test_scene.jpg"
test_img.save(test_img_path, "JPEG", quality=95)

print("Created test image:")
display(Image(test_img_path, width=300))

In [ ]:
# Generate video from image + prompt
prompt_with_image = "The robot arm moves down and grasps the apple, then lifts it up"

print(f"Generating video with image and prompt: {prompt_with_image}")
print("This may take a few minutes...")

result = client.generate_video_openai_style(
    prompt=prompt_with_image,
    image_path=test_img_path
)

if result["success"]:
    print("✅ Video generated successfully!")
    print(f"Metadata: {result.get('metadata', {})}")
    
    # Download and display
    video_path = "/tmp/example2_video.mp4"
    if client.download_video(result['full_url'], video_path):
        print(f"Video saved to: {video_path}")
        display(Video(video_path, width=400))
else:
    print(f"❌ Generation failed: {result['error']}")

## Example 3: Using Direct API

In [ ]:
# Use the direct API for more control
direct_prompt = "A cat walking across a kitchen counter, knocking over a plant"

print(f"Using direct API with prompt: {direct_prompt}")

result = client.generate_video_direct(
    prompt=direct_prompt,
    # You can specify model parameters here
    # model_size="14B",
    # num_gpus=1
)

if result["success"]:
    print("✅ Video generated with direct API!")
    print(f"Video path on server: {result['video_path']}")
    print(f"Model size used: {result.get('model_size', 'default')}")
    
    # Download and display
    video_path = "/tmp/example3_video.mp4"
    if client.download_video(result['full_download_url'], video_path):
        print(f"Video saved to: {video_path}")
        display(Video(video_path, width=400))
else:
    print(f"❌ Generation failed: {result['error']}")

## Example 4: Interactive Prompt Input

In [ ]:
# Interactive example - you can modify this cell to try different prompts

# Some example prompts to try:
example_prompts = [
    "A paper airplane flying through a sunny office",
    "Rain falling on a window while coffee steams in a mug",
    "A butterfly landing on a flower in a garden",
    "A toy car driving around obstacles on a desk",
    "Leaves falling from a tree in autumn wind"
]

print("Example prompts you can try:")
for i, prompt in enumerate(example_prompts, 1):
    print(f"{i}. {prompt}")

# Modify this line to use your own prompt:
custom_prompt = example_prompts[0]  # Change index or replace with your own prompt

print(f"\nUsing prompt: {custom_prompt}")
print("Generating...")

start_time = time.time()
result = client.generate_video_openai_style(prompt=custom_prompt)
generation_time = time.time() - start_time

if result["success"]:
    print(f"✅ Generated in {generation_time:.1f} seconds!")
    
    video_path = "/tmp/custom_video.mp4"
    if client.download_video(result['full_url'], video_path):
        file_size = os.path.getsize(video_path) / (1024 * 1024)  # MB
        print(f"Video saved: {video_path} ({file_size:.1f} MB)")
        display(Video(video_path, width=400))
else:
    print(f"❌ Failed: {result['error']}")

## Utility: Check Generated Videos

In [ ]:
# List all generated videos in /tmp
import glob

video_files = glob.glob("/tmp/*video*.mp4") + glob.glob("/tmp/cosmos_*.mp4")

if video_files:
    print("Generated videos:")
    for video_file in video_files:
        size_mb = os.path.getsize(video_file) / (1024 * 1024)
        print(f"  {video_file} ({size_mb:.1f} MB)")
    
    # Display the latest one
    latest_video = max(video_files, key=os.path.getctime)
    print(f"\nMost recent video: {latest_video}")
    display(Video(latest_video, width=400))
else:
    print("No generated videos found in /tmp")

## Summary

This notebook showed how to:
1. Initialize the Cosmos Video Client
2. Generate videos from text prompts only
3. Generate videos from text + image input
4. Use both OpenAI-compatible and direct APIs
5. Download and display generated videos

**Next steps:**
- Try your own prompts and images
- Experiment with different model parameters
- Use the client in your own applications